In [103]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
from scraper import *
import numpy as np
import scipy.sparse as sp
%load_ext autoreload
%autoreload 2

# display all pandas columns
pd.set_option('display.max_columns', 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Friendship Matrix

Scraping of initiative tables

In [4]:
df_business = pd.read_csv('data/Business.csv')
df_role = pd.read_csv('data/BusinessRole.csv')
df_status = pd.read_csv('data/BusinessStatus.csv')
df_type = pd.read_csv('data/BusinessType.csv')
df_member = pd.read_csv('data/MemberCouncil.csv')

/Users/joachimmuth/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Explanation

* **Business** is the table of initiatives
* **BusinessRole** table of all the person who act for or against a business. IMPORTANT! **BusinessRole.MemberCouncilNumber** link to **MemberCouncil.PersonNumber**. **BusinessShortNumber** link businesses and roles.
    * Auteur
    * Cosignataire
    * Porte-parole
    * Opposant(e)
    * Repreneur
    
* **BusinessType** contain all 18 types of businesses. As the type is reapeated in **Business.BusinessType** and **Business.BusinessTypeName**, it is not useful for us.
* **BusinessStatus** give the current status of a business. Again, this information is given in **Business.BusinessStatusText**, so not useful


In [34]:
print(df_business.shape)
df_business.head(2)

(38802, 44)


,Unnamed: 0,BusinessShortNumber,BusinessStatus,BusinessStatusDate,BusinessStatusText,BusinessType,BusinessTypeAbbreviation,BusinessTypeName,Category,Description,DocumentationText,DraftText,FederalCouncilProposal,FederalCouncilProposalDate,FederalCouncilProposalText,FederalCouncilResponseText,FirstCouncil1,FirstCouncil1Abbreviation,FirstCouncil1Name,FirstCouncil2,FirstCouncil2Abbreviation,FirstCouncil2Name,ID,InitialSituation,IsLeadingDepartment,Language,Modified,MotionText,Proceedings,ReasonText,ResponsibleDepartment,ResponsibleDepartmentAbbreviation,ResponsibleDepartmentName,SubmissionCouncil,SubmissionCouncilAbbreviation,SubmissionCouncilName,SubmissionDate,SubmissionLegislativePeriod,SubmissionSession,SubmittedBy,SubmittedText,TagNames,Tags,Title
0,0,11.758,27,2000-09-26T00:00:00,Liquidé,3,Iv.ct.,Initiative cantonale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19110758,NaN,True,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,4.0,DFI,Département de l'intérieur,NaN,NaN,NaN,1973-08-15T00:00:00,39.0,4311,Berne,<text><p>La législation fédérale doit être dév...,NaN,NaN,Médicaments. Législation
1,1,78.222,27,1981-03-10T00:00:00,Liquidé,4,Iv.pa.,Initiative parlementaire,NaN,Rapport de la Commission du 27.08.1979,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19780222,NaN,NaN,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,1978-06-05T00:00:00,40.0,4311,NaN,NaN,NaN,NaN,Code pénal. Interruption de la grossesse (Gira...


In [46]:
print(df_role.shape)
df_role.head()

(356969, 19)


,Unnamed: 0,BusinessNumber,BusinessShortNumber,BusinessSubmissionDate,BusinessTitle,BusinessType,BusinessTypeAbbreviation,BusinessTypeName,CantonNumber,CommitteeNumber,ID,IdExternal,Language,MemberCouncilNumber,Modified,ParlGroupNumber,ReturnType,Role,RoleName
0,0,19110758,11.758,1973-08-15T00:00:00,Médicaments. Législation,3,Iv.ct.,Initiative cantonale,2.0,NaN,8cf23579-9fd7-4be3-a338-b654da878d8c,NaN,FR,NaN,2010-12-26T13:07:38.53,NaN,1,7,Auteur
1,1,19850227,85.227,1985-02-07T00:00:00,Droit des assurances sociales,4,Iv.pa.,Initiative parlementaire,NaN,NaN,f2ccb9d9-b2fd-4fda-abeb-43e61f020921,NaN,FR,148.0,2010-12-26T13:07:38.53,NaN,0,7,Auteur
2,2,19880208,88.208,1988-11-25T00:00:00,Abandon du projet de centrale nucléaire de Graben,3,Iv.ct.,Initiative cantonale,11.0,NaN,4c34e2a2-891e-4b28-8917-170c4c3d2313,NaN,FR,NaN,2010-12-26T13:07:38.53,NaN,1,7,Auteur
3,3,19890245,89.245,1989-11-27T00:00:00,Service civil,4,Iv.pa.,Initiative parlementaire,NaN,NaN,4156c078-6a3c-4279-95f7-4802374379a6,NaN,FR,114.0,2010-12-26T13:07:38.53,NaN,0,7,Auteur
4,4,19890255,89.255,1989-12-15T00:00:00,Réserves d'or de la Banque nationale,4,Iv.pa.,Initiative parlementaire,NaN,NaN,081932bd-7632-4ef4-8b51-a66ee174be98,NaN,FR,102.0,2010-12-26T13:07:38.53,NaN,0,7,Auteur


## Get number of initiative the guy are author

In [92]:
member_number_initiatives  = df_role.loc[(df_role.MemberCouncilNumber.notnull()) & (df_role.RoleName == 'Auteur')].groupby('MemberCouncilNumber').size()

member_number_initiatives = member_number_initiatives.to_frame(name='Times_author').head()

In [96]:
member_number_initiatives.head()

,Times_author
MemberCouncilNumber,
1.0,42
2.0,6
6.0,1
7.0,10
8.0,5


## SciPy sparse matrix

Create a Scipy relation with attribute:
* row: autor
* column: co-signer
* data: number he co-sign for him

We can also do it with a pandas, and limite the number of person for juste the one still in function. Anyway...

In [217]:
# create Autor and Co-signer table

Cosignataire = df_role.loc[(df_role.MemberCouncilNumber.notnull()) & (df_role.RoleName == 'Cosignataire')]
Cosignataire = Cosignataire[['BusinessNumber', 'MemberCouncilNumber']]
Cosignataire = Cosignataire.astype(int)
print("Cosignataires: ", Cosignataire.shape)

Auteur = df_role.loc[(df_role.MemberCouncilNumber.notnull()) & (df_role.RoleName == 'Auteur')]
Auteur = Auteur[['BusinessNumber', 'MemberCouncilNumber']]
Auteur = Auteur.astype(int)
print("Auteurs: ", Auteur.shape)
Auteur.head(2)

Cosignataires:  (316548, 2)
Auteurs:  (31095, 2)


,BusinessNumber,MemberCouncilNumber
1,19850227,148
3,19890245,114


In [218]:
# Fill the sparse matrix

max_id = df_member.PersonNumber.max()
friends = sp.lil_matrix((max_id, max_id), dtype=np.int32)

def add_to_friend(autor, coautors):
    for coautor in coautors:
        friends[autor, coautor] += 1
        
        
def fill_matrix(Auteur, Cosignataire):
    for (auteur_num, bus_num) in zip(Auteur.MemberCouncilNumber, Auteur.BusinessNumber):
        coautors = Cosignataire.loc[Cosignataire.BusinessNumber == bus_num]['MemberCouncilNumber']
        if coautors.size != 0:
            add_to_friend(auteur_num, coautors)
        
fill_matrix(Auteur, Cosignataire)
friends.nonzero()

(array([   1,    1,    1, ..., 4209, 4209, 4209], dtype=int32),
 array([   1,    8,    9, ..., 4200, 4202, 4208], dtype=int32))

In [252]:
# return a pandas.DataFrame with the friend of the person, ordered by times they cosign something

def get_friends(friend_matrix, person_num):
    x = friend_matrix[person_num,:].nonzero()[1]
    y = friend_matrix[person_num,:].data[0]

    df = pd.DataFrame({'Person_num':x, 'times_cosigner':y })
    df = df.sort_values(by='times_cosigner', ascending=False)
    df = df.reset_index(drop=True)
    return df
   
get_friends(friends, 1).head()

,Person_num,times_cosigner
0,246,32
1,56,30
2,183,29
3,25,28
4,39,27
